In [17]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from core import criar_modelo, ParImageGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

In [18]:
tf.config.run_functions_eagerly(True)

os.makedirs('../model', exist_ok=True)
os.makedirs('../results', exist_ok=True)

In [19]:
# ============================
# 🔁 Carregar todos os dados
# ============================
gen_temporario = ParImageGenerator('../image/treinamento', batch_size=1, augmentacao=False)
dados = list(zip(gen_temporario.imagens, gen_temporario.labels))  # manter como lista

In [20]:
# ============================
# 🔁 K-Fold Cross-Validation
# ============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
fold = 1
accuracies = []
losses = []

log_path = '../results/cv_info.txt'
with open(log_path, 'w') as log_file:

    for train_idx, val_idx in kf.split(dados):
        log_file.write(f"[Fold {fold}]\n")
        log_file.write(f"Treino: {list(train_idx)}\n")
        log_file.write(f"Validação: {list(val_idx)}\n\n")

        print(f"\n🌀 Treinando Fold {fold}...")

        train_data = [dados[i] for i in train_idx]
        val_data = [dados[i] for i in val_idx]

        train_gen = ParImageGenerator(dados=train_data, batch_size=8, augmentacao=True)
        val_gen = ParImageGenerator(dados=val_data, batch_size=8, augmentacao=False)

        modelo = criar_modelo()
        modelo.compile(optimizer=tf.keras.optimizers.Adam(),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

        checkpoint = ModelCheckpoint(
            filepath=f'../model/melhor_modelo_fold{fold}.h5',
            monitor='val_accuracy',
            save_best_only=True, # salva o melhor modelo
            verbose=1
        )

        logger = CSVLogger(f'../results/historico_treinamento_fold{fold}.csv', append=False)

        modelo.fit(
            train_gen,
            validation_data=val_gen,
            epochs=15,
            # steps_per_epoch=len(train_gen),
            steps_per_epoch=2,
            callbacks=[checkpoint, logger],
            verbose=1
        )

        loss, accuracy = modelo.evaluate(val_gen, verbose=1)
        print(f"📊 [Fold {fold}] Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        accuracies.append(accuracy)
        losses.append(loss)
        fold += 1


🌀 Treinando Fold 1...


c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9688 - loss: 0.5060
Epoch 1: val_accuracy improved from -inf to 0.81250, saving model to ../model/melhor_modelo_fold1.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 43s 36s/step - accuracy: 0.9583 - loss: 0.5339 - val_accuracy: 0.8125 - val_loss: 6.1204
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.8750 - loss: 0.6695 
Epoch 2: val_accuracy did not improve from 0.81250
2/2 ━━━━━━━━━━━━━━━━━━━━ 63s 49s/step - accuracy: 0.8750 - loss: 0.6984 - val_accuracy: 0.8036 - val_loss: 1187.7991
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9062 - loss: 0.2736 
Epoch 3: val_accuracy did not improve from 0.81250
2/2 ━━━━━━━━━━━━━━━━━━━━ 61s 49s/step - accuracy: 0.9167 - loss: 0.2740 - val_accuracy: 0.8125 - val_loss: 6975.3721
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7188 - loss: 1.6863 
Epoch 4: val_accuracy improved from 0.81250 to 0.82143, saving model to ../model/melhor_modelo_fold1.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 78s 66s/step - accuracy: 0.7083 - loss: 1.7407 - val_accuracy: 0.8214 - val_loss: 59.5401
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7188 - loss: 0.4361 
Epoch 5: val_accuracy did not improve from 0.82143
2/2 ━━━━━━━━━━━━━━━━━━━━ 61s 49s/step - accuracy: 0.7083 - loss: 0.4577 - val_accuracy: 0.1964 - val_loss: 1103.2620
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.6562 - loss: 1.0706 
Epoch 6: val_accuracy did not improve from 0.82143
2/2 ━━━━━━━━━━━━━━━━━━━━ 61s 49s/step - accuracy: 0.6667 - loss: 1.0021 - val_accuracy: 0.1964 - val_loss: 1752.5912
Epoch 7/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.6875 - loss: 0.5329 
Epoch 7: val_accuracy did not improve from 0.82143
2/2 ━━━━━━━━━━━━━━━━━━━━ 65s 51s/step - accuracy: 0.7083 - loss: 0.5191 - val_accuracy: 0.1964 - val_loss: 1460.2592
Epoch 8/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 1.0000 - loss: 0.2868 
Epoch 8: val_accuracy did not improve from 0.8214

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 26s/step - accuracy: 0.4583 - loss: 0.8378 - val_accuracy: 0.8750 - val_loss: 1.4543
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 1.0000 - loss: 0.0065 
Epoch 2: val_accuracy did not improve from 0.87500
2/2 ━━━━━━━━━━━━━━━━━━━━ 61s 49s/step - accuracy: 1.0000 - loss: 0.0061 - val_accuracy: 0.8661 - val_loss: 23.5412
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8125 - loss: 0.8247 
Epoch 3: val_accuracy did not improve from 0.87500
2/2 ━━━━━━━━━━━━━━━━━━━━ 58s 47s/step - accuracy: 0.7917 - loss: 0.8694 - val_accuracy: 0.8750 - val_loss: 38.4345
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.9062 - loss: 0.1861 
Epoch 4: val_accuracy improved from 0.87500 to 0.88393, saving model to ../model/melhor_modelo_fold3.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 48s/step - accuracy: 0.9167 - loss: 0.1863 - val_accuracy: 0.8839 - val_loss: 428.7019
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.8750 - loss: 0.2820 
Epoch 5: val_accuracy did not improve from 0.88393
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 48s/step - accuracy: 0.8750 - loss: 0.2907 - val_accuracy: 0.8661 - val_loss: 48679.4961
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8125 - loss: 0.5413 
Epoch 6: val_accuracy did not improve from 0.88393
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 47s/step - accuracy: 0.8333 - loss: 0.4930 - val_accuracy: 0.8661 - val_loss: 3252749.2500
Epoch 7/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.6875 - loss: 1.6044 
Epoch 7: val_accuracy did not improve from 0.88393
2/2 ━━━━━━━━━━━━━━━━━━━━ 64s 53s/step - accuracy: 0.7083 - loss: 1.8005 - val_accuracy: 0.8661 - val_loss: 76009024.0000
Epoch 8/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8750 - loss: 0.3606 
Epoch 8: val_accuracy did not improve fr

2/2 ━━━━━━━━━━━━━━━━━━━━ 91s 72s/step - accuracy: 0.4583 - loss: 0.6031 - val_accuracy: 0.8929 - val_loss: 1.8478
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9062 - loss: 0.3638 
Epoch 2: val_accuracy did not improve from 0.89286
2/2 ━━━━━━━━━━━━━━━━━━━━ 80s 66s/step - accuracy: 0.9167 - loss: 0.3399 - val_accuracy: 0.8839 - val_loss: 20.5896
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9688 - loss: 0.1190
Epoch 3: val_accuracy did not improve from 0.89286
2/2 ━━━━━━━━━━━━━━━━━━━━ 56s 49s/step - accuracy: 0.9583 - loss: 0.1520 - val_accuracy: 0.8929 - val_loss: 211.1096
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9688 - loss: 0.1964 
Epoch 4: val_accuracy did not improve from 0.89286
2/2 ━━━━━━━━━━━━━━━━━━━━ 97s 79s/step - accuracy: 0.9583 - loss: 0.2443 - val_accuracy: 0.8839 - val_loss: 438.8500
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.8438 - loss: 1.2544 
Epoch 5: val_accuracy did not improve from 0.89286
2/2 ━

2/2 ━━━━━━━━━━━━━━━━━━━━ 89s 70s/step - accuracy: 0.5417 - loss: 0.7243 - val_accuracy: 0.8750 - val_loss: 0.9779
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.8750 - loss: 0.1249 
Epoch 2: val_accuracy did not improve from 0.87500
2/2 ━━━━━━━━━━━━━━━━━━━━ 47s 31s/step - accuracy: 0.8750 - loss: 0.1278 - val_accuracy: 0.8750 - val_loss: 1.3943
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7188 - loss: 2.0931 
Epoch 3: val_accuracy did not improve from 0.87500
2/2 ━━━━━━━━━━━━━━━━━━━━ 66s 54s/step - accuracy: 0.7500 - loss: 1.8615 - val_accuracy: 0.8750 - val_loss: 12.0278
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.8438 - loss: 0.6433 
Epoch 4: val_accuracy did not improve from 0.87500
2/2 ━━━━━━━━━━━━━━━━━━━━ 69s 56s/step - accuracy: 0.8333 - loss: 0.6071 - val_accuracy: 0.8750 - val_loss: 533.9283
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7500 - loss: 0.6221 
Epoch 5: val_accuracy did not improve from 0.87500
2/2 ━

2/2 ━━━━━━━━━━━━━━━━━━━━ 74s 55s/step - accuracy: 1.0000 - loss: 0.2458 - val_accuracy: 0.8839 - val_loss: 1262.6525
Epoch 9/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.9375 - loss: 0.4093 
Epoch 9: val_accuracy did not improve from 0.88393
2/2 ━━━━━━━━━━━━━━━━━━━━ 77s 64s/step - accuracy: 0.9167 - loss: 0.4977 - val_accuracy: 0.8839 - val_loss: 20953.2871
Epoch 10/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9375 - loss: 0.3016 
Epoch 10: val_accuracy did not improve from 0.88393
2/2 ━━━━━━━━━━━━━━━━━━━━ 67s 55s/step - accuracy: 0.9167 - loss: 0.3780 - val_accuracy: 0.8750 - val_loss: 41835.3555
Epoch 11/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7500 - loss: 1.9031 
Epoch 11: val_accuracy did not improve from 0.88393
2/2 ━━━━━━━━━━━━━━━━━━━━ 79s 65s/step - accuracy: 0.7500 - loss: 1.7981 - val_accuracy: 0.8750 - val_loss: 19648.9492
Epoch 12/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9375 - loss: 0.5054 
Epoch 12: val_accuracy did not improve 

In [22]:
# ============================
# 📈 Resultados finais
# ============================
media_acc = np.mean(accuracies)
media_loss = np.mean(losses)

In [23]:
print(f"\n✅ Cross-validation finalizada!")
print(f"📉 Média da Loss: {media_loss:.4f}")
print(f"✅ Média da Accuracy: {media_acc:.4f}")


✅ Cross-validation finalizada!
📉 Média da Loss: 120635.5401
✅ Média da Accuracy: 0.4286


In [24]:
# Salvar resumo no log
with open(log_path, 'a') as log_file:
    log_file.write("=== Resultado Final ===\n")
    log_file.write(f"Média da Loss: {media_loss:.4f}\n")
    log_file.write(f"Média da Accuracy: {media_acc:.4f}\n")